<a href="https://colab.research.google.com/github/marcusborela/Sir-ChatGPT/blob/main/code/Sir_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import os
import tarfile
import io


## Rotinas utilitárias

In [117]:
def mostra_dict(dicionario: dict):
    """
    Imprime informações sobre o dicionário recebido como parâmetro.

    Argumentos:
    - dicionario: um dicionário a ser impresso

    Retorna:
    - None
    """
    # obtém a primeira e última chave do dicionário
    primeiro_elemento = list(dicionario.keys())[0]
    ultimo_elemento = list(dicionario.keys())[-1]

    # imprime o tamanho do dicionário e as informações sobre seus limites
    print(f"O dicionário tem tamanho: {len(dicionario)}")
    print(f"Seus limites:\n {primeiro_elemento}:\n {dicionario[primeiro_elemento]},\n {ultimo_elemento}:\n {dicionario[ultimo_elemento]}")


# Etapa 1: Coleta dos dados

## Baixar coleção CISI.ALL

In [2]:
import os
import urllib.request
import tarfile
from collections import defaultdict


In [3]:

# Define a URL da coleção CISI
url = "http://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz"

# Define o caminho onde o arquivo será salvo
file_path = "cisi.tar"

# Faz o download do arquivo
urllib.request.urlretrieve(url, file_path)

# Extrai os arquivos da coleção CISI
with tarfile.open(file_path, "r") as tar:
    tar.extractall()


In [4]:
os.remove('cisi.tar.gz')
os.remove('cisi.tar')

Carregando o CISI.ALL

Este código carrega o arquivo CISI.ALL em um dicionário chamado documents, onde as chaves são os IDs dos documentos e os valores são dicionários com os campos "title", "author" e "text". Por exemplo, para acessar o título do documento de ID 1, basta fazer documents[1]["title"]

In [17]:
len(doc_set)

1460

In [91]:
def carregar_cisi_arquivo(nome_arquivo:str, se_debug:bool=False):

  # abre o arquivo CISI.ALL em modo de leitura
  with open(nome_arquivo, "r") as f:
      lines = f.readlines()

  # inicializa o dicionário
  documents = {}

  # inicializa as variáveis de controle
  doc_id = None
  field = None
  text = ""

  # percorre as linhas do arquivo
  for line in lines:
    # se a linha começa com ".I", então é um novo documento
    if line.startswith((".I", ".T", ".A", ".W", ".X", ".B")):
      # faça algo se a linha começar com um desses valores
        if doc_id is not None and field != "id":
            documents[doc_id][field] = text.strip()
            if se_debug:
              print(f"Atribuido a documents[{doc_id}][{field}] = {text.strip()}")
            text = ""
    # se a linha começa com ".I", então é um novo documento
    if line.startswith(".I"):
        # extrai o ID do documento da linha
        doc_id = int(line.split()[1])
        field = "id"
        documents[doc_id] = {}
        if se_debug:
          print(f"novo doc_id {doc_id}")
    # se a linha começa com ".T", ".A" ou ".W", então é um novo campo
    elif line.startswith(".T"):
        field = "title"
    elif line.startswith(".A"):
        field = "author"
    elif line.startswith(".X"):
        field = "reference"
    elif line.startswith(".B"):
        field = "bibliograph"
    elif line.startswith(".W"):
        field = "text"
    # caso contrário, é um texto que faz parte do campo atual
    else:
        text += line
    if se_debug:
      print(f'doc_id {doc_id} field {field}: {line}')
      if doc_id == 4: break 

  # adiciona o último documento ao dicionário
  if doc_id is not None:
      documents[doc_id][field] = text.strip()

  return documents

In [92]:
documentos = carregar_cisi_arquivo('CISI.ALL')

In [119]:
mostra_dict(documentos)

O dicionário tem tamanho: 1460
Seus limites:
 1:
 {'title': '18 Editions of the Dewey Decimal Classifications', 'author': 'Comaromi, J.P.', 'text': "The present study is a history of the DEWEY Decimal\nClassification.  The first edition of the DDC was published\nin 1876, the eighteenth edition in 1971, and future editions\nwill continue to appear as needed.  In spite of the DDC's\nlong and healthy life, however, its full story has never\nbeen told.  There have been biographies of Dewey\nthat briefly describe his system, but this is the first\nattempt to provide a detailed history of the work that\nmore than any other has spurred the growth of\nlibrarianship in this country and abroad.", 'reference': '1\t5\t1\n92\t1\t1\n262\t1\t1\n556\t1\t1\n1004\t1\t1\n1024\t1\t1\n1024\t1\t1'},
 1460:
 {'title': 'Modern Integral Information Systems for Chemistry and Chemical Technology', 'author': 'Chernyi, A.I.', 'text': "At the present time, about 15% of all the world publications of \nscientific and

In [95]:
consultas = carregar_cisi_arquivo('CISI.QRY')

In [120]:
mostra_dict(consultas)

O dicionário tem tamanho: 112
Seus limites:
 1:
 {'text': 'What problems and concerns are there in making up descriptive titles?\nWhat difficulties are involved in automatically retrieving articles from\napproximate titles?\nWhat is the usual relevance of the content of articles to their titles?'},
 112:
 {'title': 'A Fast Procedure for the Calculation of Similarity Coefficients in\nin Automatic Classification', 'author': 'Willett, P.', 'text': 'A fast algorithm is described for comparing the lists of terms representing\ndocuments in automatic classification experiments.  The speed of the procedure\narises from the fact that all of the non-zero-valued coefficicents for a given\ndocument are identified together, using an inverted file to the terms in the\ndocument collection.  The complexity and running time of the algorithm are\ncompared with previously described procedures.', 'bibliograph': '(Information Processing & Management, Vol. 17, No. 2, 1981, pp. 53-60)'}


In [115]:

# inicializa o dicionário de relevância por consulta
relevancia_consulta = {}

# abre o arquivo CISI.REL em modo de leitura
with open('CISI.REL') as f:
    # percorre as linhas do arquivo
    for line in f.readlines():
        # extrai o ID da consulta e do documento da linha
        qry_id = int(line.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0])
        doc_id = int(line.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1])
        
        # adiciona o ID do documento na lista de relevância da consulta
        if qry_id in relevancia_consulta:
            relevancia_consulta[qry_id].append(doc_id)
        else:
            relevancia_consulta[qry_id] = []
            relevancia_consulta[qry_id].append(doc_id)
# Fonte de apoio: https://www.kaggle.com/code/razamh/some-changes-in-cisi-eda

In [121]:
mostra_dict(relevancia_consulta)

O dicionário tem tamanho: 76
Seus limites:
 1:
 [28, 35, 38, 42, 43, 52, 65, 76, 86, 150, 189, 192, 193, 195, 215, 269, 291, 320, 429, 465, 466, 482, 483, 510, 524, 541, 576, 582, 589, 603, 650, 680, 711, 722, 726, 783, 813, 820, 868, 869, 894, 1162, 1164, 1195, 1196, 1281],
 111:
 [328, 422, 448, 485, 503, 509]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Etapa 2: Pré-processamento dos textos

In [139]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer, PorterStemmer

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 
#stemmer = SnowballStemmer('english')


In [148]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [143]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [158]:
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer

def preprocessa_texto(texto,
                      to_lower=True, 
                      remove_pontuacao=True,
                      remove_stopwords=True,
                      aplica_stemming=True,
                      aplica_lematizacao=True):
    """
    Função que realiza o pré-processamento de um texto.
    
    Parâmetros:
    texto (str): Texto a ser pré-processado
    to_lower (bool): Flag que indica se deve transformar o texto para lower case. Default: True
    remove_pontuacao (bool): Flag que indica se deve remover a pontuação do texto. Default: True
    remove_stopwords (bool): Flag que indica se deve remover as stop words do texto. Default: True
    aplica_stemming (bool): Flag que indica se deve aplicar stemming no texto. Default: True
    aplica_lematizacao (bool): Flag que indica se deve aplicar lematização no texto. Default: True
    
    Retorna:
    str: Texto pré-processado
    """
    # Transforma o texto em lower case
    if to_lower:
        texto = texto.lower()

    # Remove pontuação
    if remove_pontuacao:
        texto = re.sub(r'[^\w\s]', '', texto)

    palavras = texto.split()

    # Remove stop words
    if remove_stopwords:
        palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in stop_words]
        palavras = palavras_sem_stopwords

    # Aplica stemming
    if aplica_stemming:
        palavras_stemizadas = [stemmer.stem(palavra) for palavra in palavras]
        palavras = palavras_stemizadas

    # Aplica lematização
    if aplica_lematizacao:
        palavras_lematizadas = [lemmatizer.lemmatize(palavra) for palavra in palavras]
        palavras = palavras_lematizadas

    return palavras


In [159]:
preprocessa_texto("This is an example of text.") 

['exampl', 'text']

In [179]:
assert preprocessa_texto("This is a simple text.") == ['simpl', 'text']
assert preprocessa_texto("Hello! My name is John. Nice to meet you!") == ['hello', 'name', 'john', 'nice', 'meet']
assert preprocessa_texto("We are learning about Natural Language Processing.") == ['learn', 'natur', 'languag', 'process']
assert preprocessa_texto("The quick brown fox jumps over the lazy dog.") == ['quick', 'brown', 'fox', 'jump', 'lazi', 'dog']
assert preprocessa_texto("To be, or not to be: that is the question.") == ["question"]
assert preprocessa_texto("I'm a developer at OpenAI. I love working with AI and NLP technologies!") == ['im', 'develop', 'openai', 'love', 'work', 'ai', 'nlp', 'technolog']
assert preprocessa_texto("The cat is on the mat.") == ['cat', 'mat']
assert preprocessa_texto("An investment in knowledge pays the best interest.") == ['invest', 'knowledg', 'pay', 'best', 'interest']
assert preprocessa_texto("The quick brown fox jumps over the lazy dog.") == ['quick', 'brown', 'fox', 'jump', 'lazi', 'dog']
assert preprocessa_texto("To be, or not to be: that is the question.") == ["question"]
assert preprocessa_texto("I'm a developer at OpenAI. I love working with AI and NLP technologies!") == ['im', 'develop', 'openai', 'love', 'work', 'ai', 'nlp', 'technolog']
assert preprocessa_texto("The cat is on the mat.") == ['cat', 'mat']
assert preprocessa_texto("An investment in knowledge pays the best interest.") == ['invest', 'knowledg', 'pay', 'best', 'interest']


In [180]:
assert preprocessa_texto("Hello World!") == ['hello', 'world']
assert preprocessa_texto("Hello, World!!!") == ['hello', 'world']
assert preprocessa_texto("Hello World", remove_pontuacao=False) == ['hello', 'world']
assert preprocessa_texto("Hello World", remove_stopwords=False) == ['hello', 'world']
assert preprocessa_texto("I am running in the park", aplica_stemming=False) == ['running', 'park']
assert preprocessa_texto("I am running in the park", to_lower=False, remove_stopwords=False, aplica_stemming=False, aplica_lematizacao=False) == ['I', 'am', 'running', 'in', 'the', 'park']
assert preprocessa_texto("I am running in the park", aplica_lematizacao=False) == ['run', 'park']
assert preprocessa_texto("I am running in the park", to_lower=False) == ['i', 'run', 'park']


In [181]:
documentos[1]

{'title': '18 Editions of the Dewey Decimal Classifications',
 'author': 'Comaromi, J.P.',
 'text': "The present study is a history of the DEWEY Decimal\nClassification.  The first edition of the DDC was published\nin 1876, the eighteenth edition in 1971, and future editions\nwill continue to appear as needed.  In spite of the DDC's\nlong and healthy life, however, its full story has never\nbeen told.  There have been biographies of Dewey\nthat briefly describe his system, but this is the first\nattempt to provide a detailed history of the work that\nmore than any other has spurred the growth of\nlibrarianship in this country and abroad.",
 'reference': '1\t5\t1\n92\t1\t1\n262\t1\t1\n556\t1\t1\n1004\t1\t1\n1024\t1\t1\n1024\t1\t1'}

In [184]:
def preprocessa_texto_em_dict(parm_dict, 
                      to_lower=True, 
                      remove_pontuacao=True,
                      remove_stopwords=True,
                      aplica_stemming=True,
                      aplica_lematizacao=True):
  """
  Recebe um dicionário e retorna uma cópia do mesmo com os valores da chave "text" pré-processados.

  Args:
  parm_dict (dict): Dicionário com chaves de texto e valores em texto.
  to_lower (bool): Transforma o texto em caixa baixa. Padrão é True.
  remove_pontuacao (bool): Remove a pontuação do texto. Padrão é True.
  remove_stopwords (bool): Remove as palavras de parada do texto. Padrão é True.
  aplica_stemming (bool): Aplica a técnica de stemming no texto. Padrão é True.
  aplica_lematizacao (bool): Aplica a técnica de lematização no texto. Padrão é True.

  Returns:
  dict: Novo dicionário com a nova chave 'texto_prep' e seus valores pré-processados.

  new_dict = dict(parm_dict)  # cria uma cópia do dicionário
  for elemento in new_dict:
    for key in new_dict[elemento]:
      if key == "text":
          new_dict[elemento]["text_prep"] = preprocessa_texto(new_dict[elemento]["text"], to_lower=to_lower, 
                                                    remove_pontuacao=remove_pontuacao,
                                                    remove_stopwords=remove_stopwords,
                                                    aplica_stemming=aplica_stemming,
                                                    aplica_lematizacao=aplica_lematizacao)
  return new_dict
  """
  new_dict = {}
  for elemento in parm_dict:
    new_dict[elemento] = parm_dict[elemento].copy()
    for key in parm_dict[elemento]:
      if key == "text":
          new_dict[elemento]["text_prep"] = preprocessa_texto(new_dict[elemento]["text"], to_lower=to_lower, 
                                                    remove_pontuacao=remove_pontuacao,
                                                    remove_stopwords=remove_stopwords,
                                                    aplica_stemming=aplica_stemming,
                                                    aplica_lematizacao=aplica_lematizacao)
  return new_dict

In [185]:
consultas_prep = preprocessa_texto_em_dict(consultas)

In [186]:
consultas_prep[1]

{'text': 'What problems and concerns are there in making up descriptive titles?\nWhat difficulties are involved in automatically retrieving articles from\napproximate titles?\nWhat is the usual relevance of the content of articles to their titles?',
 'text_prep': ['problem',
  'concern',
  'make',
  'descript',
  'titl',
  'difficulti',
  'involv',
  'automat',
  'retriev',
  'articl',
  'approxim',
  'titl',
  'usual',
  'relev',
  'content',
  'articl',
  'titl']}

In [187]:
documentos_prep = preprocessa_texto_em_dict(documentos)

In [188]:
documentos_prep[1]

{'title': '18 Editions of the Dewey Decimal Classifications',
 'author': 'Comaromi, J.P.',
 'text': "The present study is a history of the DEWEY Decimal\nClassification.  The first edition of the DDC was published\nin 1876, the eighteenth edition in 1971, and future editions\nwill continue to appear as needed.  In spite of the DDC's\nlong and healthy life, however, its full story has never\nbeen told.  There have been biographies of Dewey\nthat briefly describe his system, but this is the first\nattempt to provide a detailed history of the work that\nmore than any other has spurred the growth of\nlibrarianship in this country and abroad.",
 'reference': '1\t5\t1\n92\t1\t1\n262\t1\t1\n556\t1\t1\n1004\t1\t1\n1024\t1\t1\n1024\t1\t1',
 'text_prep': ['present',
  'studi',
  'histori',
  'dewey',
  'decim',
  'classif',
  'first',
  'edit',
  'ddc',
  'publish',
  '1876',
  'eighteenth',
  'edit',
  '1971',
  'futur',
  'edit',
  'continu',
  'appear',
  'need',
  'spite',
  'ddc',
  'long',

# Etapa 3: Indexação

In [ ]:
Já apliquei o pré-processamento nos documentos e nas consultas da CISI Collection.

Nossa próxima etapa é a indexação (com BM25) para posterior recuperação.

Preciso de uma orientação sua para o que fazer agora?